# Exploring `build_renewable_profiles` outputs

The code below initialise the to be used packages. Further, `_sets_path_to_root` makes sure that we set the path to your pypsa-africa folder which we need for reading files. Please, also make sure you set up the `toast` environment which is shared as `environment-max.yml` in the envs folder.

In [ ]:
import atlite
import cartopy.crs as ccrs
import xarray as xr
import geopandas as gpd
import holoviews as hv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import pypsa
import shutil
from rasterio.plot import show
from atlite.gis import shape_availability, ExclusionContainer

import os
import sys
sys.path.append('../')  # to import helpers
from scripts._helpers import _sets_path_to_root
_sets_path_to_root("pypsa-africa")


plt.rcParams['figure.figsize'] = [7, 7]
%matplotlib inline

## Let's open the profiles
The `build_renewable_profiles` for each technology are stored in the `resources` folder. We read first the path and open then with xarray the .nc file. As you can see there is quite a lot data available.  What you should note is that only the `potential` is grided (can be shown in a nice raster on a map). From the `profiles` we can only retrieve timeseries data without mapping it.

In [ ]:
res_t_path = os.path.realpath("resources")+"/profile_solar.nc"
res_t = xr.open_dataset(res_t_path)

In [ ]:
res_t

### The renewable potential in [MW] per cutout.

In [ ]:
res_t["potential"].plot(vmin=1820, vmax=res_t.potential[:].max())

Let's zoom-in. For that purpose we needs the boundaries, in this case of Nigeria. With `geopandas` we retrieve the geographic shapes (in vector format) for Nigeria and read the shape boundaries.

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
africa = world.query('continent == "Africa"')
countries = ["Nigeria"] # "Niger"]
shapes = world[world.name.isin(countries)].set_index('name')


import math

# For showing cutouts in Africa
# shapes = africa
shapes.plot();
print(shapes.total_bounds)
latmin, lonmin, latmax, lonmax = shapes.total_bounds

#latmin, lonmin, latmax, lonmax = math.floor(latmin), math.ceil(lonmin), math.floor(latmax), math.ceil(lonmax)

Now we need to add the boundaries to the plot. Note that we also adjusted the y-scale to better show the difference accross the regions.

In [ ]:
res_t = res_t.sel(x=np.arange(latmin,latmax,0.01), y=np.arange(lonmin,lonmax,0.01), method="nearest")

In [ ]:
res_t["potential"].plot(vmin=1820, vmax=res_t.potential[:].max())
# ax.set_xlim(0,20)
# ax.set_ylim(0,10)

## Timeseries
Timeseris plots are helpful to analyse the seasonality and general variability. Below we show how you could plot some of these.

In [ ]:
res_t['profile'].sum(dim='bus').plot()

In [ ]:
res_t['profile'].sel(bus="4358", time=slice("2013-06-01", "2013-06-4")).plot()

In [ ]:
res_t["profile"].isel(bus=sum(range(10)), time=range(0,1000)).plot()